In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

In [12]:
# 14, 6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5, 16
SUBJECT_ID_TEST = [6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5]
SUBJECT_ID_TEST = ['S'+str(x) for x in SUBJECT_ID_TEST]
subject_id_test = 'AD1'

##### READ DATASET #####
# DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/WESAD'
# NAME_DATASET = DATA_DIR.split('/')[-1]

# data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_groups_1.npy')
# data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_ground_truth_1.npy')
# data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_stats_feats_1.npy')
# data_ft_con = np.load(f'/home/nmduy/STRESS/NewCode/EmbededFt/{NAME_DATASET}_WRIST_contrastive_embed_{subject_id_test}.npy')
# # with open('test.npy', 'wb') as f:
# #     np.save(f, np.array([1, 2]))

DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/AffectiveROAD_Data/Database'
NAME_DATASET = 'AffectiveROAD'
data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_groups_1.npy')
data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_ground_truth_1.npy')
data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_stats_feats_1.npy')

if NAME_DATASET == 'AffectiveROAD':
    indices = np.where(data_gt >= 0)[0]
    data_ft = data_ft[indices]
    data_group = data_group[indices]
    data_gt = data_gt[indices]
data_ft_con = np.load(f'/home/nmduy/STRESS/NewCode/EmbededFt/{NAME_DATASET}/{NAME_DATASET}_contrastive_embed_LH_{subject_id_test}.npy')

# Create dataframe for dataset
column_values = [f'f{x}' for x in range(data_ft_con.shape[1])]
data_full = pd.DataFrame(data = data_ft_con,  
                         columns = column_values)
data_full['subject_id'] = data_group
data_full['label'] = data_gt
list_subject_id = np.unique(data_full['subject_id']).tolist()

In [4]:
data_ft_con.shape

(34625, 128)

In [19]:
data_test = data_full[data_full.subject_id == subject_id_test]
data_train = data_full[data_full.subject_id != subject_id_test]

# split test sets
X_test = data_test.iloc[:,:-2].to_numpy()
y_test = data_test.iloc[:,-1].to_numpy()

# split into train - validate
X_train = data_train.iloc[:,:-2].to_numpy()
y_train = data_train.iloc[:,-1].to_numpy()

# Scaler Data
# scaler = StandardScaler()
# scaler = RobustScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [20]:
# clf = RandomForestClassifier(n_estimators = 1000, random_state = 0, 
#                              n_jobs = -1, max_features='sqrt', 
#                              oob_score=True, bootstrap=True, class_weight = 'balanced')

clf = SVC(kernel='rbf', random_state=0, 
          class_weight='balanced', C=10)

In [21]:
clf.fit(X_train, y_train)

SVC(C=10, class_weight='balanced', random_state=0)

In [22]:
# Prediction
Y_pred_test = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)
acc_test = accuracy_score(y_test, Y_pred_test)
acc_train = accuracy_score(y_train, Y_pred_train)

f1_test = f1_score(y_test, Y_pred_test)
f1_train = f1_score(y_train, Y_pred_train)

bacc_test = balanced_accuracy_score(y_test, Y_pred_test)
bacc_train = balanced_accuracy_score(y_train, Y_pred_train)

In [9]:
bacc_train, bacc_test

(0.7132727576126319, 0.5136195279149577)

In [10]:
f1_train, f1_test

(0.6869275334174839, 0.519781718963165)

In [23]:
bacc_train, bacc_test

(0.7940407354261647, 0.3953136770737423)

In [24]:
f1_train, f1_test

(0.7619594061536256, 0.3809523809523809)